In [1]:
%load_ext autoreload

In [2]:
%autoreload 
import logging
import os
import pathlib
import torch
from ssd.engine.inference import do_evaluation
from ssd.config.defaults import cfg
from ssd.utils.logger import setup_logger
from train import start_train

In [3]:
config_file = "configs/mnist_tdt4265_server.yaml"

cfg.merge_from_file(config_file)
cfg.freeze()
output_dir = pathlib.Path(cfg.OUTPUT_DIR)
output_dir.mkdir(exist_ok=True, parents=True)

logger = setup_logger("SSD", output_dir)

logger.info("Loaded configuration file {}".format(config_file))
with open(config_file, "r") as cf:
    config_str = "\n" + cf.read()
    logger.info(config_str)
logger.info("Running with config:\n{}".format(cfg))


2021-03-23 14:14:48,032 SSD INFO: Loaded configuration file configs/mnist_tdt4265_server.yaml
2021-03-23 14:14:48,034 SSD INFO: 
MODEL:
    NUM_CLASSES: 11
    THRESHOLD: 0.7
    BACKBONE:
        NAME: 'improved'
        PRETRAINED: False
        OUT_CHANNELS: [512, 1024, 512, 512, 256, 256]
        INPUT_CHANNELS: 3
    PRIORS:
        MIN_SIZES: [[10, 10], [60, 60], [111, 111], [162, 162], [213, 213], [264, 264]]
        MAX_SIZES: [[60, 60], [111, 111], [162, 162], [213, 213], [264, 264], [315, 315]]
        ASPECT_RATIOS: [[2], [2, 3], [2, 3], [2, 3], [2], [2]]
        BOXES_PER_LOCATION: [4, 6, 6, 6, 4, 4]
INPUT:
    IMAGE_SIZE: [300, 300]
    PIXEL_MEAN: [123.675, 116.280, 103.530]
    PIXEL_STD: [1, 1, 1]
DATASETS:
    TRAIN: ("mnist_detection_train", "mnist_detection_val")
    TEST: ("mnist_detection_val", )
SOLVER:
    MAX_ITER: 15000
    GAMMA: 0.1
    BATCH_SIZE: 16
    LR: 5e-4
OUTPUT_DIR: 'outputs/improved'
DATASET_DIR: "/work/datasets"

2021-03-23 14:14:48,036 SSD INFO: 

In [4]:
model = start_train(cfg)

Detector initialized. Total Number of params:  36.15M
Backbone number of parameters: 33.94M
SSD Head number of parameters: 2.21M
2021-03-23 14:14:50,180 SSD.trainer INFO: Loading checkpoint from outputs/improved/model_final.pth
2021-03-23 14:14:50,392 SSD.trainer INFO: Loading optimizer from outputs/improved/model_final.pth
2021-03-23 14:14:51,928 SSD.trainer INFO: Start training ...
2021-03-23 14:14:55,011 SSD.trainer INFO: Saving checkpoint to outputs/improved/model_final.pth
2021-03-23 14:14:56,265 SSD.trainer INFO: Total training time: 0:00:03 (0.0002 s / it)


In [5]:
logger.info('Start evaluating...')
torch.cuda.empty_cache()  # speed up evaluating after training finished
do_evaluation(cfg, model)

2021-03-23 14:14:56,296 SSD INFO: Start evaluating...
2021-03-23 14:14:56,439 SSD.inference INFO: Evaluating mnist_detection_val dataset(1000 images):


100%|██████████| 100/100 [00:13<00:00,  7.57it/s]


2021-03-23 14:15:10,319 SSD.inference INFO: mAP: 0.9018
0               : 0.9078
1               : 0.8646
2               : 0.9052
3               : 0.9064
4               : 0.9060
5               : 0.9075
6               : 0.9061
7               : 0.9026
8               : 0.9083
9               : 0.9036



[{'metrics': {'mAP': 0.9018027776597384,
   '0': 0.9077993342757572,
   '1': 0.8646305974344708,
   '2': 0.905155716519353,
   '3': 0.9064256275225915,
   '4': 0.9060076588216525,
   '5': 0.9074993633817164,
   '6': 0.9060624302669842,
   '7': 0.902554578829434,
   '8': 0.9082682655134315,
   '9': 0.903624204031994}}]